In [ ]:
import azureml
from azureml.core import Workspace, Experiment, Datastore, Environment
from azureml.core.runconfig import RunConfiguration
from azureml.data.datapath import DataPath, DataPathComputeBinding
from azureml.data.data_reference import DataReference
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.core import Pipeline, PipelineData, PipelineParameter
from azureml.pipeline.steps import PythonScriptStep, EstimatorStep
from azureml.widgets import RunDetails
from azureml.train.estimator import Estimator
import os

print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
# The following is needed if Azure ML SDK is old 
# !pip install --upgrade azureml-sdk

## Get workspace and compute

In [ ]:
# workspace
ws = Workspace.from_config()
print(ws.name[:4], ws.resource_group[:4], ws.location, ws.subscription_id[:10], sep = '\n')

In [ ]:
computetarget="cpu-cluster"
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=computetarget)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(
        vm_size='STANDARD_DS3_V2',
        vm_priority='lowpriority',
        min_nodes=0, 
        max_nodes=2,
        idle_seconds_before_scaledown=600
    )
    cpu_cluster = ComputeTarget.create(ws, computetarget, compute_config)

cpu_cluster.wait_for_completion(show_output=True)
compute = ws.compute_targets[computetarget]

print('Compute registered: ', compute)

## Create python step that downloads the model

In [ ]:
# we just need azureml-sdk
# note that we can .amlignore the requirement.txt file cause the contents are added
# in the run configuration
pythonEnvironment = Environment.from_pip_requirements('myEnv', 'requirements.txt')
run_config = RunConfiguration()
run_config.environment = pythonEnvironment
# Fix warning about remote compute supporting only docker
run_config.environment.docker.enabled = True

downloadModelStep = PythonScriptStep(
    'download_model.py',
    source_directory='.',
    name='Download model test',
    compute_target=compute,
    runconfig=run_config
)


## Create and publish the Pipeline

In [ ]:
from azureml.pipeline.core import PublishedPipeline

pipeline_name="Download model pipeline"

# Disable previously registered pipelines
published_pipeline = PublishedPipeline.list(workspace=ws)
for x in published_pipeline:
    if (x.name == pipeline_name):
        x.disable()
        print("Disabled pipeline with id ", x.id, " and name ", x.name)
    else:
        print("Skipping pipeline with id ", x.id, " and name ", x.name)

In [ ]:
pipeline = Pipeline(workspace=ws, steps=[downloadModelStep])

published_pipeline = pipeline.publish(
    name=pipeline_name, 
    description="Test to download model")

In [ ]:
# Submit the pipeline to be run
pipeline_run = Experiment(ws, 'test-model-download',).submit(published_pipeline)
print('Run created with ID: ', pipeline_run.id)